# Store kitti sequence

Create a sequence

In [37]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
#os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [38]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/07/image_0'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/07/image_1'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 5.49 ms, sys: 12 µs, total: 5.5 ms
Wall time: 4.73 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [ ]:
seq = Sequence()
for f in ProgressBar()(frame[:]):
    seq.add_keyframe(f,run_ba = False)
    
#set matching keyframes
pose_frame(frame[1060],frame[1])

for obs in frame[1060].get_observations():
    # obs matches previous keyframe
    if obs.has_mappoint():
        obs.check_mappoint()
        obs.register_mappoint()
        
                
    

  0% (1 of 1101) |                        | Elapsed Time: 0:00:00 ETA:  0:08:07

connected mappoints  0
7768 
 [[  9.99978553e-01  -3.93679436e-04   6.53745681e-03   6.44901477e-03]
 [  3.95020461e-04   9.99999901e-01  -2.03839478e-04  -1.29580590e-03]
 [ -6.53737592e-03   2.06417536e-04   9.99978610e-01  -9.14791802e-02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


  0% (2 of 1101) |                        | Elapsed Time: 0:00:00 ETA:  0:08:32

connected mappoints  103
7769 
 [[  9.99976283e-01  -7.88058927e-04   6.84191313e-03   1.01656787e-02]
 [  7.87211992e-04   9.99999682e-01   1.26478445e-04   2.24734267e-03]
 [ -6.84201063e-03  -1.21089410e-04   9.99976586e-01  -9.74268420e-02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


  0% (3 of 1101) |                        | Elapsed Time: 0:00:01 ETA:  0:08:58

...

connected mappoints  102
8383 
 [[  9.99999154e-01  -8.33508132e-04  -9.98772866e-04   1.36933055e-02]
 [  8.32674895e-04   9.99999305e-01  -8.34385990e-04   1.83879529e-02]
 [  9.99467640e-04   8.33553631e-04   9.99999153e-01  -9.25433430e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 56% (617 of 1101) |############          | Elapsed Time: 0:14:35 ETA:  0:15:17

connected mappoints  93
8384 
 [[  9.99998356e-01  -1.29842481e-03  -1.26572233e-03   1.15872610e-02]
 [  1.29813367e-03   9.99999131e-01  -2.30812955e-04   2.15027496e-02]
 [  1.26602092e-03   2.29169499e-04   9.99999172e-01  -9.15465627e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 56% (618 of 1101) |############          | Elapsed Time: 0:14:37 ETA:  0:15:06

connected mappoints  102


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [23]:
frame[4].get_world_pose()

array([[  9.99865281e-01,  -1.02289921e-03,   1.63821029e-02,
          2.15890400e-02],
       [  9.89241849e-04,   9.99997384e-01,   2.06249206e-03,
          2.78449093e-03],
       [ -1.63841698e-02,  -2.04600834e-03,   9.99863677e-01,
         -2.01220248e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])

In [29]:
seq.keyframes[0].get_world_pose()

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

IndexError: list index out of range

In [16]:
mp1 = [o for o in seq.keyframes[10].get_observations() if o.mappoint is not None]
[o.get_affine_coords() for o in mp1][:5]

IndexError: list index out of range

In [4]:
# mp1 = [o for o in seq.keyframes[10].get_observations() if o.mappoint is not None]
# [o.get_world_coords() for o in mp1][:5]

In [5]:
mp1 = [o.mappoint for o in frame[1].get_observations() if o.mappoint is not None]
[len(m.observations) for m in mp1]

[1,
 2,
 2,
 2,
 4,
 2,
 2,
 5,
 3,
 3,
 1,
 1,
 3,
 3,
 2,
 2,
 4,
 1,
 11,
 12,
 7,
 2,
 3,
 1,
 3,
 1,
 2,
 5,
 4,
 4,
 2,
 6,
 1,
 1,
 1,
 1,
 2,
 1,
 7,
 2,
 4,
 1,
 4,
 8,
 6,
 2,
 1,
 5,
 6,
 3,
 9,
 1,
 1,
 3,
 1,
 6,
 2,
 1,
 2,
 3,
 2,
 7,
 2,
 1,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 1,
 2,
 2,
 1,
 5,
 10,
 1,
 5,
 8,
 3,
 3,
 4,
 3,
 2,
 3,
 1,
 2,
 3,
 3,
 3,
 1,
 7,
 2,
 13,
 2,
 2,
 1,
 5,
 2,
 5,
 1,
 2,
 2,
 7,
 1,
 2,
 2,
 2,
 2,
 2,
 7,
 1,
 2,
 1,
 4,
 8,
 4,
 5,
 2,
 3,
 8,
 4,
 3,
 5,
 3,
 6,
 3,
 4,
 4,
 2,
 3,
 1,
 6,
 1,
 5,
 3,
 3,
 3,
 6,
 2,
 2,
 2,
 6,
 3,
 2,
 4,
 1,
 2,
 1,
 2,
 1,
 3,
 5,
 3,
 2,
 3,
 1,
 1,
 4,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 6,
 1,
 2,
 4,
 3,
 3,
 3,
 2,
 6,
 1,
 3,
 4,
 2,
 5,
 12,
 2,
 2,
 1,
 3,
 2,
 1,
 3,
 1,
 1,
 4,
 6,
 1,
 1,
 6,
 1,
 2,
 2,
 1,
 16,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 9,
 3,
 9,
 1,
 1]

In [33]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.get_world_pose().flatten() for kf in keyframes ] ])

def mappoints_to_np(mappoints):
    kfp = np.hstack([ [ [m.id] for m in mappoints ], [ m.get_cumulative_world_pose() for m in mappoints  ] ])
    return kfp

# count = 10
# poses = np.array([ np.eye(4) for f in seq.keyframes[:count] ])
# observations = [ (m.get_last_observation(), m) for m in get_mappoints( seq.keyframes[:count]) ]
# print(poses)
# obs = [o[0].get_affine_coords() for o in observations ]
                

# for i in range(count):
#     pose = seq.keyframes[i].get_pose()
#     print("old pose ", "\n" , pose)
#     for j in range(count-i):
#         newPose = np.dot( pose, poses[j] )
#         print("new pose ", "\n" , newPose, "\n", "end")
        
#         poses[j] = np.dot( pose, poses[j] )
#         for k, o in enumerate(observations):
#             if o[0].frame == seq.keyframes[i]:
#                 obs[k] = np.dot(obs[k], pose)
#                 o[1].cumulative_world_pose = obs[k]

# print("obswrvation", "\n", mappoints_ob_to_np(observations))             
                
# for i in range(len(seq.keyframes)):
    
#     f = seq.keyframes[i]
#     pose = f.get_pose()
    
#     if i > 0:
#         f.cumulative_pose = np.dot(seq.keyframes[i-1].cumulative_pose, pose)
#     else:
#         f.cumulative_pose = pose
                
                

cv_keyframes = seq.keyframes
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

print('Keyframes numpy \n', (cv_keyframes_BA[:, 2 :]), '\n')
print('Keyframes numpy \n', (mappoints[:]), '\n')
# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.fullBundleAdjustment(cv_keyframes_BA, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[: , 2:]), '\n')





# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


# keyframes_np_MO = keyframes_to_np(seq.keyframes)

np.save('cv_keyframes', cv_keyframes_BA)
np.save('f_keyframes', f_keyframes_np)
np.save('mappoints', mappoints_np)
np.save('links', links_np)


Keyframes count
 1101 

Keyframes numpy 
 [[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99978553e-01  -3.93679436e-04   6.53745681e-03 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 ..., 
 [  9.99253220e-01  -3.34536934e-02   1.93352886e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99239914e-01  -3.38986275e-02   1.92477725e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99239283e-01  -3.39478187e-02   1.91937634e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]] 

Keyframes numpy 
 [[  1.06138000e+05  -1.75112745e+02   1.96613708e+00   3.18318968e+01
    1.00000000e+00]
 [  1.22882000e+05  -1.53619471e+02   3.68622483e+00  -3.01094356e+01
    1.00000000e+00]
 [  1.43441000e+05   3.67427648e+02  -1.10898092e+02  -8.77962321e+02
    1.00

Keyframes after BA 
 [[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99975222e-01  -3.88376176e-04   7.02881267e-03 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99999723e-01  -5.59755964e-06   7.43922544e-04 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 ..., 
 [  9.99334314e-01  -3.30312006e-02   1.54876695e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99336236e-01  -3.33917894e-02   1.45628265e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99318585e-01  -3.35355426e-02   1.54186026e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]] 



In [36]:
cv_keyframes_BA[:50]

array([[  0.00000000e+00,   5.56500000e+03,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   5.56600000e+03,   9.99975222e-01,
         -3.88376176e-04,   7.02881267e-03,   6.38950501e-03,
          3.92309838e-04,   9.99999767e-01,  -5.58278119e-04,
         -1.25789186e-03,  -7.02859421e-03,   5.61021759e-04,
          9.99975142e-01,  -9.15204578e-02,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  2.00000000e+00,   5.56700000e+03,   9.99999723e-01,
         -5.59755964e-06,   7.43922544e-04,   4.00586921e-06,
          5.81400753e-06,   9.99999958e-01,  -2.90953031e-04,
          4.12205479e-06,  -7.43920884e-04,   2.90957276e-04,
      